In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import h5py
import os

from PIL import Image

def connected_components_tf(image, num_iterations=100):
    if not tf.is_tensor(image):
        raise TypeError(f"Input type is not a tf.Tensor. Got: {type(image)}")
    
    if not isinstance(num_iterations, int) or num_iterations < 1:
        raise TypeError("Input num_iterations must be a positive integer.")
    
    if len(image.shape) < 3 or image.shape[-3] != 1:
        raise ValueError(f"Input image shape must be (*, 1, H, W). Got: {image.shape}")
    
    # Reshape image to 2D if it's more than 3D
    shape = tf.shape(image)
    image = tf.reshape(image, [-1, shape[-2], shape[-3]])
    
    # Create initial labels
    B, H, W = image.shape
    labels = tf.reshape(tf.range(B * H * W, dtype=image.dtype), [B, H, W])
    mask = tf.equal(image, 1)
    
    # Initialize labels as zero where mask is False
    labels = tf.where(mask, labels, tf.zeros_like(labels))
    
    for _ in range(num_iterations):
        # Max pool current labels to simulate the dilation effect
        labels = tf.expand_dims(labels, axis=3)  # Add a channel dimension
        pooled_labels = tf.nn.max_pool2d(labels, ksize=3, strides=1, padding='SAME')
        labels = tf.squeeze(pooled_labels, axis=[-1])  # Remove the channel dimension
        labels = tf.where(mask, labels, tf.zeros_like(labels))
    
    return tf.reshape(labels, shape)

# # Example usage
# img = tf.random.uniform((2, 1, 4, 5), dtype=tf.float32)
# img_labels = connected_components_tf(img, num_iterations=100)



2024-04-26 13:05:19.368983: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 13:05:19.401600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 13:05:19.401630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 13:05:19.402588: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 13:05:19.408110: I tensorflow/core/platform/cpu_feature_guar

In [2]:

def stitch_windows_tf(windows, k, cropx, cropy):
    # Example of adapting the stich_windows for TensorFlow
    # Assumes 'windows' is a TensorFlow tensor
    row0 = tf.concat([windows[0, 0][:-k, :-k]] +
                     [win[:-k, k:-k] for win in windows[0, 1:-1]] +
                     [windows[0, -1][:-k, k:]], axis=1)
    rows = [row0]
    for r in range(1, windows.shape[0] - 1):
        rows.append(tf.concat([windows[r, 0][k:-k, :-k]] +
                              [win[k:-k, k:-k] for win in windows[r, 1:-1]] +
                              [windows[r, -1][k:-k, k:]], axis=1))
    row_last = tf.concat([windows[-1, 0][k:, :-k]] +
                         [win[k:, k:-k] for win in windows[-1, 1:-1]] +
                         [windows[-1, -1][k:, k:]], axis=1)
    final = tf.concat([row0] + rows[1:] + [row_last], axis=0)
    final = final[:cropx, :cropy]
    return final

In [3]:
import tensorflow as tf

class LocatorTF:
    def __init__(self, fastrcnn_model, process_stride=64, method='max', dark_threshold=20,
                 locating_model=None, mode='static', **kwargs):
        self.fastrcnn_model = fastrcnn_model
        # self.device = device
        self.process_stride = process_stride
        self.method = method
        self.dark_threshold = dark_threshold
        self.locating_model = locating_model
        self.mode = mode
        self.p_list = kwargs.get('p_list', [8, 6, 1.5, 1, 50])
        self.meanADU = kwargs.get('meanADU', 241.0)
        self.dynamic_thres = kwargs.get('dynamic_thres', True)
        self.pretune_thresholding = kwargs.get('pretune_thresholding')

    def model_tune(self, arr):
        # Placeholder for model tuning logic
        pass

    def images_to_window_lists(self, inputs):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        outputs = []
        h, w = inputs.shape[1], inputs.shape[2]

        if self.process_stride is None:
            windows = tf.image.resize(inputs, [h * 2, w * 2], method='nearest')
            outputs.extend(tf.reshape(windows, [-1, h * 2, w * 2, 1]))
        else:
            # Add padding if necessary
            pad_height = (self.process_stride - h % self.process_stride) % self.process_stride
            pad_width = (self.process_stride - w % self.process_stride) % self.process_stride
            inputs_padded = tf.pad(inputs, [[0, 0], [0, pad_height], [0, pad_width], [0, 0]])

            # Extract patches
            windows = tf.image.extract_patches(
                images=inputs_padded,
                sizes=[1, self.process_stride, self.process_stride, 1],
                strides=[1, self.process_stride, self.process_stride, 1],
                rates=[1, 1, 1, 1],
                padding='VALID'
            )

            # Reshape output for further processing
            batch_size, num_rows, num_cols, _ = windows.shape
            outputs.extend(tf.reshape(windows, [batch_size * num_rows * num_cols, self.process_stride, self.process_stride, 1]))
        return outputs

    def predict_sequence(self, inputs):
        # self.fastrcnn_model.training = False
        counted_list = []
        eventsize_all = []

        image_cell_list = self.images_to_window_lists(inputs)
        for i, image_cell in enumerate(image_cell_list):
            if self.mode == 'dynamic_window':
                self.model_tune(image_cell)

            image_cell = (image_cell - tf.reduce_min(image_cell)) / (tf.reduce_max(image_cell) - tf.reduce_min(image_cell))
            boxes = self.fastrcnn_model.predict(image_cell)  # Assume 'predict' is implemented to handle TensorFlow inputs
            filtered_boxes = self.filter_boxes(boxes)  # Assuming a method to filter boxes based on certain criteria
            filtered, _, eventsize = self.locate(image_cell, filtered_boxes)
            counted_list.append(filtered)
            eventsize_all.extend(eventsize)

        # Stitch windows and return final image
        counted_images = stitch_windows_tf(tf.concat(counted_list, axis=0), k=3, cropx=inputs.shape[1], cropy=inputs.shape[2])
        return counted_images, eventsize_all

    def locate(self, image_array, boxes):
        width = 10
        filtered = tf.zeros_like(image_array)
        coor = []
        eventsize = []

        for box in boxes:
            y1, x1, y2, x2 = tf.cast(box, tf.int32)
            xarea = image_array[y1:y2+1, x1:x2+1]

            patch = tf.pad(xarea, [[1, 1], [1, 1]], mode='CONSTANT', constant_values=0)
            patch = tf.image.resize(patch, [width, width], method='nearest')

            if self.method == 'max':
                idx = tf.argmax(tf.reshape(patch, [-1]))
                model_y, model_x = divmod(idx, width)
            elif self.method in ['com', 'binary_com']:
                if self.method == 'binary_com':
                    patch = tf.cast(patch >= 30, tf.float32)
                coords = tf.stack(tf.meshgrid(tf.range(width), tf.range(width), indexing='ij'), axis=-1)
                total_mass = tf.reduce_sum(patch)
                center_mass = tf.reduce_sum(patch[:, :, tf.newaxis] * tf.cast(coords, tf.float32), axis=[0, 1])
                model_y, model_x = tf.unstack(tf.round(center_mass / total_mass))

            cx = model_y + y1 - 1
            cy = model_x + x1 - 1

            if cx >= 0 and cy >= 0 and cx < image_array.shape[0] and cy < image_array.shape[1]:
                filtered[cx, cy] += 1
                coor.append((cx, cy))
                eventsize.append(tf.reduce_sum(patch > 20))

        return filtered, coor, eventsize

    def filter_boxes(self, boxes):
        # Example filter that selects boxes based on area criteria
        return [box for box in boxes if 0 < (box[2] - box[0]) * (box[3] - box[1]) < 900]


In [4]:
import cv2
import numpy as np
import torch
from scipy.ndimage import center_of_mass, maximum_position
from scipy.ndimage import label, find_objects
from sklearn.metrics import pairwise_distances_argmin_min


##########################################################################################################################
### Counting methods ###
# Below list six different counting methods, primaryly use connected component labeling(CCL) to find the clusters, 
# and assign the entry position to max intensity pixel, or center of mass, or center of mass after binarization, or random.
# the last one, fastrcnn_predict, is using the ML model, instead of CCL.
# all methods return the 256x256 counted image and coords array of shape(num, 2)


def cca(img):
  '''
  only returns the stats from cca
  '''
  thresh = np.array(img > 20).astype('int8')
  output = cv2.connectedComponentsWithStatsWithAlgorithm(thresh, 8, cv2.CV_32S, 0) 
  (_, _, stats, centroids) = output
  return stats


def counting_filter_binary_com(image, threshold=20, structure = np.ones((3,3))):
    image_binary = image > threshold  # more readable
    all_labels, num = label(image_binary, structure = np.ones((3,3)))  # get blobs
    m=np.ones(shape=all_labels.shape)
    obj = center_of_mass(m, all_labels, range(1,num))
    obj = np.rint(obj).astype(int)
    x = np.zeros(shape=np.shape(image))
    x[obj[:,0],obj[:,1]]=1
    return x, obj


def counting_filter_com(image, threshold=20, structure = np.ones((3,3))):
    image_binary = image > threshold  # more readable
    all_labels, num = label(image_binary, structure = np.ones((3,3)))  # get blobs
    # m=np.ones(shape=all_labels.shape)
    obj = center_of_mass(image, all_labels, range(1,num))
    obj = np.rint(obj).astype(int)
    x = np.zeros(shape=np.shape(image))
    x[obj[:,0],obj[:,1]]=1
    return x, obj


def counting_filter_max(image, threshold=20, structure = np.ones((3,3))):
    eventsize = []
    image_binary = image > threshold  # more readable
    all_labels, num = label(image_binary, structure = np.ones((3,3)))  # get blobs
    m=np.ones(shape=all_labels.shape)
    obj = maximum_position(image, all_labels, range(1,num))
    obj = np.rint(obj).astype(int)
    x = np.zeros(shape=np.shape(image))
    x[obj[:,0],obj[:,1]]=1
    for i in np.arange(num)[1:]:
      eventsize.append(np.where(all_labels==i)[0].shape[0])
    return x, obj, np.array(eventsize).astype('int')


def counting_filter_random(image, threshold=20, structure = np.ones((3,3))):
    image_binary = image > threshold  # more readable
    all_labels, num = label(image_binary, structure = np.ones((3,3)))  # get blobs
    obj = find_objects(all_labels)
    coords = []
    for i in range(len(obj)):
      coords.append((np.random.randint(obj[i][0].start,obj[i][0].stop),
                    np.random.randint(obj[i][1].start,obj[i][1].stop)))
    coords = np.array(coords)
    x = np.zeros(shape=np.shape(image))
    x[coords[:,0], coords[:,1]] = 1
    return x, coords


def fastrcnn_predict(model, arr, process_stride, mode, **kwargs):
  from CountingNN.locator import Locator
  x = arr[None, ...]
  # device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  counting = LocatorTF(model, process_stride, 'max', 30, None, mode, meanADU = kwargs.get('meanADU'), 
                     p_list=kwargs.get('p_list'), dynamic_thres = kwargs.get('dynamic_thres'), pretune_thresholding = kwargs.get('pretune_thresholding'))
  filtered, event_sizes =  counting.predict_sequence(x)
  filtered = filtered[0]
  all_coords = []
  for value in range(1, 1 + filtered.max()):
      coords = np.array(np.where(filtered==value))        
      all_coords.append([coords]*value)
  all_coords = np.hstack(np.array(all_coords)[0]).T

  return filtered, all_coords, event_sizes

##########################################################################################################################
### Evaluation metrics calculation ###

def pos_deviation(coords, truth, threshold):
    """
    Cal the root mean square error between detected electron incident positions and the ground truth positions in units of pixels.
    """
    # elements in pair 1 need to be no less than pair 2 
    distances = []
    if len(coords):
      assigment,distances = pairwise_distances_argmin_min(coords, truth)

    return distances


def general_evaluation(file, algorithm, repeat, savepath, **kwargs):
  '''
  This function is for calculating the scores for each data file, Stack***.npz together. 
  Arguments
  -----------
  file: the validation data file. e.g., my Stack000.npz contains images with sparsity 0~0.002, array X is the input images, 
  it has the shape of [N, M, 256, 256], N different sparsity ranging from sparsitymin(0) to sparsitymax(0.002), M copies of each same sparsity.
  Then Stack001.npz contains images with sparsity 0.002~0.004, and so on. 

  algorithm: run evaluation of one counting algorithm, string of the function name defined above.

  repeat: set the number of images with identical sparsity in each data file to be used.  
  '''
  data = np.load(file)

  X = data['X'][:,:repeat]
  y = data['y'][:,:repeat]
  print('Max pixel value in ground truth:', y.max())
  # creat blank arrays to store the score values, which has the same first two dimension as X. 
  dce = np.zeros(X.shape[:2]) # dce: simple detector conversion efficiency,  the ratio of input and detected electron counts
  mae = np.zeros(X.shape[:2]) # mae: mean absolute error, the absolute error of electron counts averaged over all pixels in a single image
  nume = np.zeros(X.shape[:2]) # number of actrual electrons in the image
  recall = np.zeros(X.shape[:2]) # recall, true positive / (true positives + false negtives)
  precision = np.zeros(X.shape[:2]) # precision, true positive / (true positives + false positives)
  filtered =  np.zeros(X.shape) # i.e. the counted image

  # saving the coordinate, position deviation and event size for each detected electron event in the image, 
  # so need to create an array of objects, and the object is a list
  coords = [ [0] * X.shape[1] ] * X.shape[0]
  deviations = [ [0] * X.shape[1] ] * X.shape[0]
  eventsizes = [ [0] * X.shape[1] ] * X.shape[0]
  coords = np.array(coords, dtype=object)
  deviations = np.array(deviations, dtype=object)
  eventsizes = np.array(eventsizes, dtype=object)
  save_e_size = True


  # Now go through the NxM images to get the scores
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
          
      if algorithm =='fastrcnn_predict':
        model = kwargs.get('model')
        method = kwargs.get('method')
        stride = kwargs.get('stride')
        mode = kwargs.get('mode')
        meanADU = kwargs.get('meanADU')
        p_list = kwargs.get('p_list')
        dynamic_thres = kwargs.get('dynamic_thres')
        pretune_thresholding = kwargs.get('pretune_thresholding')
        # by using the "eval", the long string has been read as a line of code, and it runs the algorithm function
        res = eval(algorithm +"(model, X[i][j], stride, mode, meanADU=meanADU, p_list=p_list, dynamic_thres = dynamic_thres,pretune_thresholding = pretune_thresholding )")
        filtered[i,j] = res[0]
        coords[i][j] = res[1]
        # if the algorithm returns eventsize, set to save it.
        try:
          eventsizes[i][j] = res[2]
        except:
          save_e_size = False

      else: 
        res = eval(algorithm + "(X[i][j])")
        filtered[i,j] = res[0]
        coords[i][j] = res[1]
        try:
          eventsizes[i][j] = res[2]
        except:
          save_e_size = False
      
      # Get all the ground truth coordinates of electron events
      # For a pixel value 2 for example, indicating 2 electrons here, so we need to add its coordinate twice. 
      truth = []
      for value in range(1, 1+int(y[i,j].max())):
        truth_ = np.array(np.where(y[i,j]==value))        
        truth.append([truth_]*value)
      truth = np.hstack(np.array(truth)[0]).T

      total_pixel = filtered[i,j].shape[0] * filtered[i,j].shape[1]
      mae[i,j] = np.sum(np.abs(filtered[i,j]-y[i,j]))/total_pixel
      dce[i,j] = np.sum(filtered[i,j])/np.sum(y[i,j])
      nume[i,j] = np.sum(y[i,j])

      tp = 0 
      # count how many electron events are well identified, i.e., count the true positives
      for n, value in enumerate(filtered[i,j].ravel()):

        if (value != 0) & (y[i,j].ravel()[n] != 0):
          tp = tp + np.min((value, y[i,j].ravel()[n])) # multi-class considered

      recall[i,j] = tp/nume[i,j]
      precision[i,j] = tp/np.sum(filtered[i,j])

      deviations[i][j] = pos_deviation(coords[i][j], truth, 6)
      dce[i,j] = len(deviations[i][j])/np.sum(y[i,j])

  path = savepath + file[-12:-4]
  if save_e_size:
    np.savez(path+'_result.npz', coordinates = coords, result = filtered, mae = mae, dce = dce, nume = nume, 
    recall = recall, precision = precision, deviations = deviations, eventsizes = eventsizes)
  else:
    np.savez(path+'_result.npz', coordinates = coords, result = filtered, mae = mae, dce = dce, nume = nume, 
    recall = recall, precision = precision, deviations = deviations)
    
import re

import os

def create_directory_if_not_exists(directory_path):
    """
    Create a directory if it doesn't exist.

    Args:
    - directory_path (str): Path of the directory to be created.
    """
    if not os.path.exists(directory_path):
        try:
            os.makedirs(directory_path)
            print(f"Directory '{directory_path}' created successfully.")
        except OSError as e:
            print(f"Error creating directory '{directory_path}': {e}")
    else:
        print(f"Directory '{directory_path}' already exists.")

In [5]:
input_shape = (64,64,1)
num_classes = 280
num_coordinates = 4

x_input = layers.Input(shape=input_shape)
#Layer 1
x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) 
x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
#Layer 2
x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
#Layer 3
x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
#Layer 4
x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
#Layer 5
x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) 


x = layers.Flatten()(x)
# Probability output
x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)

# Bounding box output
x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)




model = tf.keras.models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
model.compile(optimizer= optimizer, loss= {'x_prob_reshape': tf.keras.losses.BinaryCrossentropy(), 'x_boxes_reshape':tf.keras.losses.MeanSquaredError()}, metrics=['accuracy'])    


2024-04-26 13:05:33.119962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21968 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [6]:
u= model.load_weights("/home/m3-learning/Documents/Research Data/Derrick's Object Detection/Models/M11overfittedmodel4variant.keras")

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 64, 64, 64)           640       ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_5 (MaxPoolin  (None, 32, 32, 64)           0         ['conv2d_10[0][0]']           
 g2D)                                                                                             
                                                                                                  
 batch_normalization_2 (Bat  (None, 32, 32, 64)           256       ['max_pooling2d_5[0][0]'

In [8]:
savepath = '/home/m3-learning/Documents/myML/ValidationResults/'
path = '/home/m3-learning/Documents/myML/ValidationData-20231107T163458Z-001/ValidationData/'

create_directory_if_not_exists(savepath)

# Regular expression pattern to match files starting with 'Stack' and ending with '.npz'
pattern = re.compile(r'^Stack.*\.npz$')

import time
start_time = time.time()
for file in os.listdir(path):
  if pattern.match(file):
    general_evaluation(path + file, algorithm = None, repeat = 1, savepath = savepath, model = u,
                      method = 'max',  mode = 'dynamic_window', stride = 64) #meanADU = 1007
    print('\nWorked on ', file, 'cost', " %s seconds." % (time.time() - start_time))
    start_time = time.time()

Directory '/home/m3-learning/Documents/myML/ValidationResults/' already exists.
Max pixel value in ground truth: 2.0


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:


# class LocatorTF:
#     def __init__(self, fastrcnn_model, process_stride=64, method='max', dark_threshold=20, locating_model=None,
#                  mode='static', **kwargs):
#         self.fastrcnn_model = fastrcnn_model
#         self.mode = mode
#         self.process_stride = process_stride
#         self.method = method
#         self.locating_model = locating_model
#         self.dark_threshold = dark_threshold
#         self.p_list = kwargs.get('p_list', [8, 6, 1.5, 1, 50])
#         self.meanADU = kwargs.get('meanADU', 241.0)
#         self.dynamic_thres = kwargs.get('dynamic_thres', True)
#         self.pretune_thresholding = kwargs.get('pretune_thresholding')




#     def model_tune(self, arr):
#         meanADU = self.meanADU * 4  # mean ADU * upsample_factor^2
#         offset = 0
#         limit = int(tf.reduce_sum(arr) / meanADU + offset)
#         arr_t = tf.cast(arr[None, None, ...] > 30, tf.float32)
        
#         # Assuming a custom implementation for connected components that returns a count
#         # since TensorFlow doesn't have a direct equivalent.
#         limit_cca = connected_components_tf(arr_t, num_iterations=10)
#         limit = max(limit_cca, limit)
#         limit = max(limit, 1)

#         self.fastrcnn_model.rpn._pre_nms_top_n = {'training': limit * self.p_list[0], 'testing': limit * self.p_list[0]}
#         self.fastrcnn_model.rpn._post_nms_top_n = {'training': limit * self.p_list[1],
#                                                    'testing': limit * self.p_list[1]}
#         self.fastrcnn_model.roi_heads.detections_per_img = int(limit * self.p_list[2])
#         # self.fastrcnn_model.roi_heads.score_thresh = self.p_list[3] / limit if limit < self.p_list[4] else 0
#         self.fastrcnn_model.roi_heads.score_thresh = self.p_list[3] / limit

#         self.fastrcnn_model.roi_heads.nms_thresh = 0.02  # smaller, delete more detections

#         if limit > (0.005 * arr.shape[0] * arr.shape[1]) and self.dynamic_thres:  # 0.002 is minimum for model13
#             self.dark_threshold = 0  # for image that not quite sparse, lift the pre-thresholding.